In [11]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


# Configure an LLM
llm = ChatOpenAI(
    model="qwen3-32b",
    temperature=0.5,
    base_url=os.environ.get("COMPATIBLE_BASE_URL"),
    api_key=os.environ.get("COMPATIBLE_API_KEY"),
    streaming=True,
    extra_body={"enable_thinking": False},
)

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.
    passage:
    {input}"""
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

structured_llm = llm.with_structured_output(Classification)

In [13]:
input = "I'm incredibly glad I met you! I think we'll be great friends!"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='positive', aggressiveness=1, language='English')

In [14]:
input = "真高兴认识你！我想我们会成为很好的朋友！"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='positive', aggressiveness=1, language='Chinese')

In [15]:
response.model_dump()

{'sentiment': 'positive', 'aggressiveness': 1, 'language': 'Chinese'}

In [16]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(..., enum=["chinese", "english", "japanese"])

structured_llm = llm.with_structured_output(Classification)

In [17]:
input = "真高兴认识你！我想我们会成为很好的朋友！"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='happy', aggressiveness=1, language='chinese')

In [18]:
input = "你真是我遇到过的最差劲的人了！"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='sad', aggressiveness=5, language='chinese')

In [19]:
input = "あなたに会えて本当に嬉しいです！きっと良い友達になれると思います！"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='happy', aggressiveness=1, language='japanese')

In [20]:
input = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": input})
response = structured_llm.invoke(prompt)
response

Classification(sentiment='happy', aggressiveness=1, language='japanese')

In [21]:
from typing import Optional

class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

structured_llm = llm.with_structured_output(Person)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)


text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

In [22]:
class People(BaseModel):
    """Extracted data about people."""

    people: list[Person]

structured_llm = llm.with_structured_output(People)

text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

People(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])